In [13]:
## Load Dataframe
import pandas as pd
import warnings
import fnmatch
import os

pd.set_option('future.no_silent_downcasting', True)
root = ".dataDir"

## Method to pivot state information for dsr-ticket/assessment-id
def aggregate_states(df, id_col):
    df['id'] = df['env']+"_"+df['tenant']+"_"+df[id_col].astype(str)
    df = df.drop('tenant', axis=1)
    df = df.drop(id_col, axis=1)
    df = df.groupby(['id','action']).agg({'timestamp': "max"}).reset_index()
    df.sort_values(by="id", ascending=True, inplace=True)
    df = df.pivot(index=['id'], columns='action', values='timestamp').reset_index()
    return df

def loadDataFrameFromFileRegex(root, regex):
    df_arr = []
    for path, subdirs, files in os.walk(root):
        for name in files:
            if fnmatch.fnmatch(name, regex):
                df = pd.read_csv(os.path.join(path, name))
                df['env']  = getFirstLevelDirName(root, path)
                df_arr.append(df)
    if not df_arr:
        warnings.warn("No matching file found in "+root+" for regex: "+regex+". Empty dataframe will be returned." )
        return pd.DataFrame()    
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=FutureWarning)      
        return pd.concat(df_arr, ignore_index=True)

def getFirstLevelDirName(root, path):
    start = path.find(root)+len(root)+1
    end = path.find("/", start)
    return path[start: end]
# df_unstruc = loadDataFrameFromFileRegex(root, 'UNSTRUCTURED-*.csv')
# df_struc = loadDataFrameFromFileRegex(root, 'STRUCTURED-*.csv')
# df_dsr = aggregate_states(loadDataFrameFromFileRegex(root, 'DSR-*.csv'), 'dsr_ticket')
# df_assmt = aggregate_states(loadDataFrameFromFileRegex(root, 'ASSESSMENT-*.csv'), 'assessment_id')
df_applianceNodeCpuMax = loadDataFrameFromFileRegex(root, 'securiti_appliance_cpu_used-max_over_time-*.csv').groupby(['appliance_id','node_id', 'ts']).agg({'value': "max"}).reset_index()


In [12]:
display(df_applianceNodeCpuMax)


,appliance_id,node_ip,ts,value
0,0af83074-d88b-4990-b033-92f28b161d2c,10.12.92.126,1722729600,7.10
1,0af83074-d88b-4990-b033-92f28b161d2c,10.12.92.126,1722733200,8.76
2,0af83074-d88b-4990-b033-92f28b161d2c,10.12.92.126,1722736800,7.08
3,0af83074-d88b-4990-b033-92f28b161d2c,10.12.92.126,1722740400,9.50
4,0af83074-d88b-4990-b033-92f28b161d2c,10.12.92.126,1722744000,7.86
...,...,...,...,...
3403,f13a3fd0-b648-43e8-a3f7-44205d533a97,10.12.92.143,1722798000,5.57
3404,f13a3fd0-b648-43e8-a3f7-44205d533a97,10.12.92.143,1722801600,7.45
3405,f13a3fd0-b648-43e8-a3f7-44205d533a97,10.12.92.143,1722805200,5.81
3406,f13a3fd0-b648-43e8-a3f7-44205d533a97,10.12.92.143,1722808800,10.61


In [ ]:

display(df_dsr[df_dsr['id'].str.contains('chipot')])
display(df_unstruc[df_unstruc['tenant'].str.contains('exl')])

In [ ]:
## Run Aggregations
df = df_unstruc.groupby(['ds','tenant','pod']).agg({'dataScannedInGB': 'sum', 'processingTimeinHrs':'sum'})
df['AvgDataInGBperdayByDw']=df.dataScannedInGB*24/df.processingTimeinHrs
df = df.add_suffix('_Count').reset_index()
df.describe(percentiles=[0.25, 0.75, 0.90])

In [ ]:
## Create Plots
import plotly.express as px
fig = px.box(df, x='ds', y='AvgDataInGBperdayByDw_Count', color='ds', points=False, log_y=True)
fig.update_layout(
    font=dict(family="Didact Gothic"),
    yaxis_title="<b>GB/day/dw<b>",
    xaxis_title="<b>fileFormat<b>",
    title_x=0.5,
    title_font=dict(size=24,color='black'),
    plot_bgcolor='rgba(0, 0, 0, 0)', #'white',  
    paper_bgcolor='rgba(0, 0, 0, 0)', #'white',
    width=1200,  
    height=900
)
fig.update_traces(quartilemethod="exclusive")
fig.update_yaxes(nticks=10, minor=dict(showgrid=True, gridwidth=2, gridcolor='Black'))
fig.show()